In [1]:
import json
from os import path
import pprint
import random

import geopandas as gpd
import matplotlib
import numpy as np
import pysal
from tqdm import tqdm, tqdm_notebook

# Set random seeds for reproducibility
random.seed(45202)
np.random.seed(45202)

# Set up PrettyPrinter
pp = pprint.PrettyPrinter()

# Force matplotlib to plot from notebook
%matplotlib inline
# Increase default plot size
matplotlib.rcParams['figure.figsize'] = (20.0, 20.0)

In [2]:
# import the cleaned neighborhood data and address naming bug with .rename()
neighborhoods = gpd.read_file(
    path.join('..', 'maps', 'clean', 'neighborhoods.shp')
).rename(columns={'neighborho': 'neighborhood'})
neighborhoods_without_border = neighborhoods.query('neighborhood != "Border"').reset_index(drop=True)

In [3]:
# Get contiguity weights matrix from shapefile. I argue queen contiguity doesn't
# make sense, as shared borders are necessary for moving between neighborhoods.

# Get weights for neighborhoods on the region edge
border_w = pysal.weights.Rook.from_dataframe(neighborhoods)
neighborhoods_on_border = border_w.neighbors[0]

# Get weights for neighborhoods inside region
neighborhood_w = pysal.weights.Rook.from_dataframe(neighborhoods_without_border)

with open('neighbors.json', 'w') as file:
    json.dump(neighborhood_w.neighbors, file)

In [4]:
# Generate 1000 random, contiguous regions/districts using PySAL
rand_districts = pysal.region.Random_Regions(
    neighborhood_w.id_order, 
    num_regions=3, 
    contiguity=neighborhood_w,
    compact=True,
    permutations=1000
)

# Create a placeholder for the generated population
initial_population = []

# Loop over feasible solutions to generate starting population object
for solution in tqdm_notebook(rand_districts.solutions_feas, desc='Generate population'):
    
#     # Create a placeholder chromosome - I've been going back and forth 
#     # on this. A list might be more appropriate, but I'm nervous about
#     # unexpected indexing behaviour, so explicit keys feel a little 
#     # safer.
#     chromosome = dict.fromkeys(range(0, neighborhoods.shape[0]), np.NaN)
    
#     # Loop through the districts
#     for idx, region in enumerate(solution.regions):
#         # Add each neighborhood in the district to the chromosome
#         for neighborhood in region:
#             chromosome[neighborhood] = idx
#     initial_population.append(chromosome)

    
    # Alternatively, construct an empty list of the length of the neighborhoods
    chromosome = [None] * neighborhoods_without_border.shape[0]
    
    for idx, region in enumerate(solution.regions):
        # Add each neighborhood in the district to the chromosome
        for neighborhood in region:
            chromosome[neighborhood] = idx
    initial_population.append(chromosome)        
    
    
#     # Or, even more alternatively, create a new object to store relevant information
#     individual = {
#         'regions': solution.regions,
#         'chromosome': [None] * neighborhoods.shape[0]
#     }

#     for idx, region in enumerate(solution.regions):
#         # Add each neighborhood in the district to the chromosome
#         for neighborhood in region:
#             individual['chromosome'][neighborhood] = idx
#     initial_population.append(individual)

Widget Javascript not detected.  It may not be installed or enabled properly.


In [5]:
# Isolate the border geometry for neighbor-checking
border = neighborhoods.query('neighborhood == "Border"')

# Create placeholder for final population
final_population = []

# Check each individual solution for districts surrounded entirely by another districts
for individual in tqdm_notebook(initial_population, desc='Check feasibility/validity'):
    
    # Set the districts to the generated values
    neighborhoods_without_border.district = individual
    
    # Create a full district map to calculate neighbors
    districts = neighborhoods_without_border.append(border).dissolve(by='district')
    
    # Get the neighbor weights from the districts
    district_w = pysal.weights.Rook.from_dataframe(districts)
    
    # Create placeholder count for surrounded districts
    surrounded_districts = 0
    for district, neighbors in enumerate(district_w):
        # If a district has one or fewer neighbours, it is invalid
        if len(neighbors[1]) <= 1: surrounded_districts = surrounded_districts + 1
    
    # If there were no surrounded districts, append to the final population
    if surrounded_districts == 0: final_population.append(individual)

        
print('Valid individual solutions:', len(final_population))

Widget Javascript not detected.  It may not be installed or enabled properly.



('Valid individual solutions:', 941)


In [7]:
# Of the valid solutions, randomly sample 100 (without replacement)
final_population_sample = random.sample(final_population, 100)
        
# Write out population to file
with open('population.json', 'w') as file:
    json.dump(final_population_sample, file)